In [47]:
import pandas as pd
import spacy
import torch


df = pd.read_csv("contrib_from_csv.csv", sep = ",", dtype= str)

# sample d'un très petit échantillon pour cet ordi
df = df.sample(1000)

In [48]:
join_breaklines = lambda x : x.replace("\x0b"," ") 
join_breaklinesn = lambda x : x.replace("\n"," ") 

In [49]:
df["dupyo"] = df["Contribution"].apply(join_breaklines)

In [50]:
df["clean"] = df["dupyo"].apply(join_breaklinesn)

In [51]:
pd.set_option('display.max_colwidth', None)
df["clean"]

155374                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  

In [52]:
documents = df.clean.tolist()
documents

["Page 1 Le 15/01/2019 Bonjour, Serait, il possible de mettre un transport scolaire en place sur chamaux et Verney plutôt que faire déplacer toutes ces familles et leurs voitures, niveau pollution bonjour, et de verser des primes transport à chacun plutôt que de créer un emploi dans la région Pourquoi les habitants n'ont plus de botin téléphonique? Vive les écrans. Merci Valérie Perréard Le 31/01/2019",
 'Renforcer les contrôles en matière de fraude fiscale ou sociale Justice fiscale :  Pourrait-on envisager que TOUS les citoyens contribuent à l\'impôt sur le revenu en rétablissant des tranches d\'imposition supplémentaires "vers le haut vers le bas"? Peut-on considérer que les salaires ou retraites, supérieurs à 2000€ correspondent aux 20% les plus riches? Diminuer considérablement les "privilèges" accordés aux anciens président de la république et premiers ministres. Diminuer le nombre de députés et sénateurs. (Un député par département et un suppléant chargé de l\'informer des probl

In [53]:
# from sklearn.datasets import fetch_20newsgroups

# dataset = fetch_20newsgroups(shuffle=True, random_state=1, remove=('headers', 'footers', 'quotes'))
# documents = dataset.data

In [54]:
type(documents)

list

In [55]:
from spacy.lang.fr.stop_words import STOP_WORDS as fr_stop
from spacy.lang.en.stop_words import STOP_WORDS as en_stop

final_stopwords_list = list(fr_stop) + list(en_stop) + ["\n"]

def remove_stopwords(text):
    stopwords = final_stopwords_list
    text = [word for word in text if word not in stopwords]
    return text

In [56]:
nostopword = remove_stopwords(documents)

In [57]:
documents[0]

"Page 1 Le 15/01/2019 Bonjour, Serait, il possible de mettre un transport scolaire en place sur chamaux et Verney plutôt que faire déplacer toutes ces familles et leurs voitures, niveau pollution bonjour, et de verser des primes transport à chacun plutôt que de créer un emploi dans la région Pourquoi les habitants n'ont plus de botin téléphonique? Vive les écrans. Merci Valérie Perréard Le 31/01/2019"

In [58]:
def lemmatize(docs, allowed_postags = ["NOUN", "ADJ", "VERB", "ADV"]):
  '''
  Performs lemmization of input documents.
  Args:
    - docs: list of strings with input documents
    - allowed_postags: list of accepted Part of Speech (POS) types
  Output:
    - list of strings with lemmatized input
  '''
  nlp = spacy.load("fr_core_news_md", disable = ["parser", "ner"])
  nlp.max_length = 3000000 #this is possible as we're not using parser or ner.
  lemmatized_docs = []
  for doc in docs:
    doc = nlp(doc)
    tokens = []
    for token in doc:
      if token.pos_ in allowed_postags:
        tokens.append(token.lemma_)
    lemmatized_docs.append(" ".join(tokens))
  return (lemmatized_docs)

In [59]:
lemmatized = lemmatize(documents)

In [64]:
from sklearn.decomposition import LatentDirichletAllocation
n_topics = 3

In [65]:
from sklearn.feature_extraction.text import CountVectorizer

tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2, max_features=1000, stop_words="english")
tf = tf_vectorizer.fit_transform(lemmatized)

In [66]:
# Créer le modèle LDA
lda = LatentDirichletAllocation(
        n_components=n_topics, 
        max_iter=5, 
        learning_method='online', 
        learning_offset=50.,
        random_state=0)

# Fitter sur les données
lda.fit(tf)

LatentDirichletAllocation(learning_method='online', learning_offset=50.0,
                          max_iter=5, n_components=3, random_state=0)

In [67]:
def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic {}:".format(topic_idx))
        print(" ".join([feature_names[i] for i in topic.argsort()[:-no_top_words - 1:-1]]))

no_top_words = 10
display_topics(lda, tf_vectorizer.get_feature_names_out(), 10)

Topic 0:
article décembre cahier mairie social droit doléance pourcent actif mourir
Topic 1:
ne pas plus pouvoir tout avoir faire devoir falloir citoyen
Topic 2:
illisible retraite tout plus pas salaire ne suppression mot taxe
